<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/tensorboard/Accelerator_W&B_Tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This Notebook accompanies the report titled "[Tensorboard with Accelerators - A Guide](https://wandb.ai/sauravmaheshkar/Accelerator-TensorBoard/reports/Tensorboard-with-Accelerators-A-Guide--Vmlldzo5Nzk2MzM)"

---
##### Most of the code in this notebook is ported from the [**"Transfer learning and fine-tuning"**](https://www.tensorflow.org/tutorials/images/transfer_learning) tutorial from the Tensorflow website in order to demonstrate easy integration of the wandb client into pre-existing workflows.

Author: [@SauravMaheshkar](https://twitter.com/MaheshkarSaurav)

# Packages 📦 and Basic Setup

---

## Install Packages

In [ ]:

# Clear any TensorBoard logs from previous runs
!rm -rf ./logs/

In [ ]:

## Install the latest version of wandb client 🔥🔥
!pip install -q --upgrade wandb

In [ ]:
%%capture
import os
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image_dataset_from_directory

## Project Configuration using **`wandb.config`**

In [ ]:
import wandb

# log to Weights and biases
wandb.login()

In [ ]:


# Feel free to change these and experiment !!
config = wandb.config
config.AUTOTUNE = tf.data.AUTOTUNE
config.BATCH_SIZE = 32
config.IMG_SIZE = (160, 160)
config.base_learning_rate = 0.0001
config.seed = 2021
config.randomfliptype = "horizontal_and_vertical"
config.randomrotationfactor = 0.2
config.randomcontrastfactor = 0.2
config.IMG_SHAPE = config.IMG_SIZE + (3,)
config.epochs = 20

# 💿 The Dataset

---

In this code cell we:-

* download and extract a zip file containing images
* create a `tf.data.Dataset` using `tf.keras.preprocessing.image_dataset_from_directory`



In [ ]:
# Download and extract
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

# Create path variables for training and validation images
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

# Create train and validation tf.data.Dataset's
train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=config.BATCH_SIZE,
                                             image_size=config.IMG_SIZE)

validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=config.BATCH_SIZE,
                                                  image_size=config.IMG_SIZE)

class_names = train_dataset.class_names
# Add Class Names as config to wandb
config.class_names = train_dataset.class_names

# Split the dataset
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)
train_dataset = train_dataset.prefetch(buffer_size=config.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=config.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=config.AUTOTUNE)

# ✍️ Model Architecture 

---

## 🏗 Transfer Learning
> Excerts from Datacamp Tutorials and MachineLearningMastery blogs

First things first; Transfer learning(TL) is not a machine learning model or technique; it is rather a "**design methodology**" within machine learning. The general idea of transfer learning is to use knowledge learned from tasks for which a lot of labelled data is available in settings where only little labelled data is available. Creating labelled data is expensive, so optimally leveraging existing datasets is key.

In a traditional machine learning model, the primary goal is to generalise to unseen data based on patterns learned from the training data. With transfer learning, you attempt to kickstart this generalisation process by starting from patterns that have been learned for a different task. Essentially, instead of starting the learning process from a (often randomly initialised) blank sheet, you start from patterns that have been learned to solve a different task.

Convolutional Neural Networks' features are more generic in early layers and more original-dataset-specific in later layers. Thus, we often use these as a backbone / starting point while creating new models. A common practice is to make these base models non-trainable and just learn the later layers. You might think that this will decrease the performance, but as we'll see from training. Transfer Learning is still a viable option.

## ⚖️ DenseNet121

![](https://pytorch.org/assets/images/densenet1.png)

> From PyTorch Hub

**Dense Convolutional Network (DenseNet)**, connects each layer to every other layer in a feed-forward fashion. Whereas traditional convolutional networks with $L$ layers have $L$ connections - one between each layer and its subsequent layer - our network has $L(L+1)/2$ direct connections. For each layer, the feature-maps of all preceding layers are used as inputs, and its own feature-maps are used as inputs into all subsequent layers. DenseNets have several compelling advantages: they alleviate the vanishing-gradient problem, strengthen feature propagation, encourage feature reuse, and substantially reduce the number of parameters.

In this project we'll use [**`DenseNet121`**](https://www.tensorflow.org/api_docs/python/tf/keras/applications/densenet) for training our Binary Classifier. The Model can easily be instantiated using the **`tf.keras.applications`** Module, which provides canned architectures with pre-trained weights. For more details kindly visit this [link](https://www.tensorflow.org/api_docs/python/tf/keras/applications/densenet).

In [ ]:
def get_model():

  base_model = tf.keras.applications.densenet.DenseNet121(include_top=False, 
                                                          weights='imagenet',
                                                          input_shape=config.IMG_SHAPE
  )

  preprocess_input = tf.keras.applications.densenet.preprocess_input
  
  data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode=config.randomfliptype, seed = config.seed),
    tf.keras.layers.RandomRotation(factor = config.randomrotationfactor, seed = config.seed),
    tf.keras.layers.RandomContrast(factor = config.randomcontrastfactor, seed=config.seed)
  ])

  base_model.trainable = True

  inputs = tf.keras.Input(shape=(160, 160, 3))

  x = data_augmentation(inputs)
  x = preprocess_input(x)
  x = base_model(x, training=False)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  outputs = tf.keras.layers.Dense(1)(x)
  model = tf.keras.Model(inputs, outputs)

  return model

# 🧱 + 🏗 = 🏠 Training

---

W&B automatically logs metrics from TensorBoard into dashboards. If you've got a pre-existing training workflow based around Tensorboard, switching to wandb requires just 2 lines of code.

1. `wandb.tensorboard.patch(root_logdir="...")`
2. `run = wandb.init(..., sync_tensorboard = True)`

W&B client will then automatically log all the metrics generated during training from the logs directory into it's amazing dashboard.


## You don't even have to worry about accelerator's !!!

Just create a distribute strategy for either GPUs or TPUs and **`wandb`** automatically logs all the system metrics like GPU Power Usage, GPU Memory Allocated, GPU Temp, etc.

## Easy Model Versioning with W&B Artifacts 🏪

It's no secret that training deep learning models takes a lot of time, that's why we use Weights and Biases Artifacts to store our models weights and graphs for easy reproducibility. 

W&B artifacts allows you to store different versions of your datasets and models in the cloud as Artifacts.

For example all the model files for this project can be found [here](https://wandb.ai/sauravmaheshkar/Accelerator-TensorBoard/artifacts)

In [ ]:
PROJECT_NAME = "tensorboard-accelerator-wandb"
ENTITY = None # Replace with your entity name or Team name

In [ ]:
# Point wandb client to the log dir of Tensorboard
wandb.tensorboard.patch(root_logdir="/content/logs")

# Create a wandb run to log all your metrics
run = wandb.init(project=PROJECT_NAME, entity=ENTITY, reinit=True, sync_tensorboard=True)

# Just 3 lines of code to utilize and run training on GPUs
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)

def train_model(model_dir = '/content/model_dir'):

  # Training
  history = model.fit(train_dataset,
                      validation_data=validation_dataset,
                      epochs=config.epochs,
                      callbacks=[tf.keras.callbacks.TensorBoard()])
  
  # Save the Model and Upload it to the cloud as a Artifact
  trained_model_artifact = wandb.Artifact("DenseNet121-GPU-Adadelta", type = "model", description = "Baseline DenseNet121 model trained on GPUs using Adadelta as the optimizer")
  model.save(model_dir)
  trained_model_artifact.add_dir(model_dir)
  run.log_artifact(trained_model_artifact)

  # Finish the run
  run.finish()

In [ ]:
with strategy.scope():
  model = get_model()

  model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=config.base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
  
  train_model()